In [6]:
# Make better use of Jupyter Notebook cell width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [7]:
# Import the usual suspects. Any new functions will be introduced individually for clarity.
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, auc
from sklearn.metrics import recall_score, roc_auc_score, roc_curve, classification_report
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import make_classification
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC, SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from collections import Counter, OrderedDict
from xgboost import XGBClassifier
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

import matplotlib.pyplot as plt
import seaborn as sns
from mlxtend.plotting import plot_decision_regions
%matplotlib inline

# make prettier plots
%config InlineBackend.figure_format = 'svg' 
sns.set()

In [8]:
# Load training and validation data
X_tr = pd.read_pickle('data/X_tr.pkl')
X_val = pd.read_pickle('data/X_val.pkl')
X_te = pd.read_pickle('data/X_te.pkl')

y_tr = pd.read_pickle('data/y_tr.pkl')
y_val = pd.read_pickle('data/y_val.pkl')
y_te = pd.read_pickle('data/y_te.pkl')

In [9]:
y_tr.head()

2777    1
393     1
205     1
2023    0
2951    0
dtype: int64

In [10]:
X_tr.head()

,Hispanic,White,Black,Native,Asian,Pacific,VotingAgeCitizen,IncomePerCap,Poverty,ChildPoverty,...,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment,percent_men
0,0.026000,0.405948,-0.598919,-0.194252,0.210922,0.735115,-1.747167,-0.714128,-0.149208,-0.847519,...,0.689423,0.206995,-1.154697,0.543083,0.221199,0.248489,-0.844623,-0.152798,-1.006841,0.088284
1,-0.132038,0.127748,0.136934,-0.221638,-0.220980,-0.190445,-0.552169,-0.186248,-0.315568,-0.328132,...,0.001389,0.243693,0.845883,0.285569,0.774543,-0.596601,-0.549674,-0.152798,-0.174886,-0.139230
2,3.214239,-2.013398,-0.418710,-0.098402,0.174930,0.040945,-3.033734,-1.009202,0.998675,0.986236,...,0.251583,-0.380168,0.411795,-1.209726,0.348894,-0.326172,-0.107252,-0.384263,0.622404,-0.730766
3,-0.523697,0.445691,-0.651479,1.024409,-0.328955,-0.190445,0.208263,1.300417,-0.664924,-0.815720,...,0.626874,0.280390,-1.985126,1.459692,-0.303768,0.045667,0.600624,-0.384263,-1.804131,0.479608
4,3.585285,-2.008430,-0.628953,-0.194252,-0.184988,-0.190445,-5.284425,-1.268244,1.165035,0.996836,...,0.189034,0.390483,-1.060330,-0.577990,0.746167,-0.326172,-1.021592,0.078666,-0.209551,0.329449


In [11]:
# get baseline logistic regression model:
lr = LogisticRegression(solver='lbfgs')
lr.fit(X_tr, y_tr)
f1 = f1_score(lr.predict(X_val_a), y_val_a)
recall = recall_score(lr.predict(X_val), y_val)
print('f1 score: {} \nrecall score: {}'.format(f1, recall))

f1 score: 0.698443579766537 
recall score: 0.6275229357798165


In [12]:
# get baseline random forest model
randomforest = RandomForestClassifier(n_estimators=500)
randomforest.fit(X_tr, y_tr)
f1 = f1_score(randomforest.predict(X_val), y_val)
recall = recall_score(randomforest.predict(X_val), y_val)
print('f1 score: {} \nrecall score: {}'.format(f1, recall))

f1 score: 0.9123809523809524 
recall score: 0.8870370370370371


In [13]:
randomforest.feature_importances_

array([0.04144371, 0.03913172, 0.03256293, 0.02770963, 0.04019351,
       0.01033567, 0.03290799, 0.02840773, 0.03803229, 0.03251513,
       0.03149955, 0.03326405, 0.06080876, 0.03546632, 0.03246221,
       0.03298813, 0.03037632, 0.0309128 , 0.03814418, 0.02739739,
       0.02867302, 0.03976685, 0.03024144, 0.03963486, 0.03119104,
       0.05661597, 0.0158602 , 0.0351108 , 0.04634582])

In [14]:
# get baseline decision tree
decisiontree = DecisionTreeClassifier(max_depth=5)
decisiontree.fit(X_tr, y_tr)
f1 = f1_score(decisiontree.predict(X_val), y_val)
recall = recall_score(decisiontree.predict(X_val), y_val)
print('f1 score: {} \nrecall score: {}'.format(f1, recall))

f1 score: 0.6601941747572816 
recall score: 0.6538461538461539


In [15]:
decisiontree.feature_importances_

array([0.14193425, 0.        , 0.        , 0.        , 0.0349383 ,
       0.        , 0.        , 0.        , 0.01970777, 0.01884588,
       0.02211942, 0.04398507, 0.22749221, 0.01273592, 0.        ,
       0.05439019, 0.        , 0.        , 0.        , 0.        ,
       0.01680488, 0.04855765, 0.        , 0.02994442, 0.02918616,
       0.12296265, 0.02023858, 0.05893774, 0.0972189 ])

In [16]:
# get baseline linear svm
linearsvc = LinearSVC()
linearsvc.fit(X_tr, y_tr)
f1 = f1_score(linearsvc.predict(X_val), y_val)
recall = recall_score(linearsvc.predict(X_val), y_val)
print('f1 score: {} \nrecall score: {}'.format(f1, recall))

f1 score: 0.6412940057088486 
recall score: 0.6229205175600739


/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [17]:
# get baseline svm
svc = SVC(gamma='scale', probability=True)
svc.fit(X_tr, y_tr)
f1 = f1_score(svc.predict(X_val), y_val)
recall = recall_score(svc.predict(X_val), y_val)
print('f1 score: {} \nrecall score: {}'.format(f1, recall))

f1 score: 0.7345844504021448 
recall score: 0.6748768472906403


In [18]:
# get baseline KNN
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_tr, y_tr)
f1 = f1_score(knn.predict(X_val), y_val)
recall = recall_score(knn.predict(X_val), y_val)
print('f1 score: {} \nrecall score: {}'.format(f1, recall))

f1 score: 0.7429547395388558 
recall score: 0.6580937972768532


In [19]:
# Get baseline XGBoost
xgb = GradientBoostingClassifier()
xgb.fit(X_tr, y_tr)
f1 = f1_score(xgb.predict(X_val), y_val)
recall = recall_score(xgb.predict(X_val), y_val)
print('f1 score: {} \nrecall score: {}'.format(f1, recall))

f1 score: 0.7881040892193308 
recall score: 0.7491166077738516


In [20]:
for i in sorted(zip(X_val, randomforest.feature_importances_), key = lambda x: x[1]):
    print(i)

('Pacific', 0.010335673898364588)
('FamilyWork', 0.015860199183555853)
('OtherTransp', 0.02739738575557059)
('Native', 0.02770963155201716)
('IncomePerCap', 0.028407728160521183)
('WorkAtHome', 0.02867302022087048)
('Employed', 0.030241435408841697)
('Carpool', 0.030376315166980073)
('Transit', 0.03091280120983472)
('PublicWork', 0.031191039541564995)
('Professional', 0.03149954635145361)
('Production', 0.03246220756793105)
('ChildPoverty', 0.032515126548594675)
('Black', 0.03256293243042408)
('VotingAgeCitizen', 0.03290798947825679)
('Drive', 0.032988134461815054)
('Service', 0.03326405471757841)
('Unemployment', 0.035110800502191356)
('Construction', 0.0354663189225337)
('Poverty', 0.03803228659707957)
('Walk', 0.038144184567735114)
('White', 0.03913172013256897)
('PrivateWork', 0.03963485818865127)
('MeanCommute', 0.039766848215427246)
('Asian', 0.04019350539484349)
('Hispanic', 0.041443705941258804)
('percent_men', 0.04634581551487558)
('SelfEmployed', 0.05661597301252661)
('Office

[('Hispanic', 0.041443705941258804),
 ('White', 0.03913172013256897),
 ('Black', 0.03256293243042408),
 ('Native', 0.02770963155201716),
 ('Asian', 0.04019350539484349),
 ('Pacific', 0.010335673898364588),
 ('VotingAgeCitizen', 0.03290798947825679),
 ('IncomePerCap', 0.028407728160521183),
 ('Poverty', 0.03803228659707957),
 ('ChildPoverty', 0.032515126548594675),
 ('Professional', 0.03149954635145361),
 ('Service', 0.03326405471757841),
 ('Office', 0.060808761356133174),
 ('Construction', 0.0354663189225337),
 ('Production', 0.03246220756793105),
 ('Drive', 0.032988134461815054),
 ('Carpool', 0.030376315166980073),
 ('Transit', 0.03091280120983472),
 ('Walk', 0.038144184567735114),
 ('OtherTransp', 0.02739738575557059),
 ('WorkAtHome', 0.02867302022087048),
 ('MeanCommute', 0.039766848215427246),
 ('Employed', 0.030241435408841697),
 ('PrivateWork', 0.03963485818865127),
 ('PublicWork', 0.031191039541564995),
 ('SelfEmployed', 0.05661597301252661),
 ('FamilyWork', 0.015860199183555853

# ROC Curves

In [ ]:
models = [lr, randomforest, decisiontree, svc, knn, xgb]

In [ ]:
# fpr, tpr, thresholds = roc_curve(y_val, lr.predict_proba(X_val)[:,1])

In [ ]:
plt.plot([0,1],[0,1],c='violet',ls='--')
for model in models:
    fpr, tpr, thresholds = roc_curve(y_val, model.predict_proba(X_val)[:,1])
    plt.plot(fpr, tpr,lw=2)

    print("ROC AUC score = ", roc_auc_score(y_val, model.predict_proba(X_val)[:,1]))

plt.plot([0,1],[0,1],c='violet',ls='--')
plt.xlim([-0.05,1.05])
plt.ylim([-0.05,1.05])
plt.legend(['Chance', 'Logistic Regression', 'Random Forest', 'Decision Tree', 'SVC', 'KNN', 'XGB'])
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve for Predicting Police Shootings')

In [ ]:
for item in sorted(zip(list(X_tr.columns), list(randomforest.feature_importances_)), 
                   key=lambda x: x[1]):
    print(item)

In [ ]:
for item in sorted(zip(list(X_tr.columns), list(decisiontree.feature_importances_)), 
                   key=lambda x: x[1]):
    print(item)

In [ ]:
from matplotlib.pyplot import scatter

In [ ]:
np.corrcoef(X_tr.Drive, y_tr)

In [ ]:
scatter(x=X_tr.Hispanic, y=y_tr)

# Try Random Forest with the top 10 features

In [ ]:
X_tr_top15 = X_tr.drop(labels = ['Pacific', 'FamilyWork', 'OtherTransp', 'Native', 'IncomePerCap',
                                'WorkAtHome', 'Employed', 'Employed', 'Carpool', 'PublicWork',
                                'Transit', 'Professional', 'ChildPoverty', 'VotingAgeCitizen', 
                                'Production'], axis = 1)

X_val_top15 = X_val.drop(labels = ['Pacific', 'FamilyWork', 'OtherTransp', 'Native', 'IncomePerCap',
                                'WorkAtHome', 'Employed', 'Employed', 'Carpool', 'PublicWork',
                                'Transit', 'Professional', 'ChildPoverty', 'VotingAgeCitizen', 
                                'Production'], axis = 1)

In [ ]:
# random forest with 15 predictive features
rf15 = RandomForestClassifier(n_estimators=100)
rf15.fit(X_tr_top15, y_tr)
f1 = f1_score(rf15.predict(X_val_top15), y_val)
recall = recall_score(rf15.predict(X_val_top15), y_val)
print('f1 score: {} \nrecall score: {}'.format(f1, recall))

In [ ]:
plt.plot([0,1],[0,1],c='violet',ls='--')
fpr, tpr, thresholds = roc_curve(y_val, rf15.predict_proba(X_val_top15)[:,1])
plt.plot(fpr, tpr,lw=2)

print("ROC AUC score = ", roc_auc_score(y_val, rf15.predict_proba(X_val_top15)[:,1]))

plt.plot([0,1],[0,1],c='violet',ls='--')
plt.xlim([-0.05,1.05])
plt.ylim([-0.05,1.05])
plt.legend(['Chance', 'Random Forest'])
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve for Predicting Police Shootings')

# Try GridSearch for Random Forest

In [ ]:
n_estimators = [10,100,200,300,400,500,600,1000]
criterion = ['gini', 'entropy']
param_grid = dict(n_estimators = n_estimators, criterion = criterion)
random_forest_grid_search = RandomForestClassifier()
random_forest_grid = GridSearchCV(random_forest_grid_search, param_grid, cv=5, scoring='roc_auc', verbose=2, n_jobs=-1)
random_forest_grid.fit(X_tr, y_tr)
random_forest_grid_predictions = random_forest_grid.predict(X_val)
random_forest_best_params = random_forest_grid.best_params_
random_forest_best_estimator = random_forest_grid.best_estimator_
random_forest_best_cm = confusion_matrix(y_val,random_forest_grid_predictions)
random_forest_best_cr = classification_report(y_val,random_forest_grid_predictions)
print(random_forest_best_params, random_forest_best_estimator, random_forest_best_cm, random_forest_best_cr)

In [ ]:
random_forest_y_score = random_forest_grid.predict_proba(X_val)[:, 1]
random_forest_grid_fpr, random_forest_grid_tpr, auc_thresholds = roc_curve(y_val, random_forest_y_score)
roc_auc_random_forest_grid = auc(random_forest_grid_fpr, random_forest_grid_tpr)
print(auc(random_forest_grid_fpr, random_forest_grid_tpr))

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_val, random_forest_grid.best_estimator_.predict(X_val))